In [2]:
%pip install -U spacy

Note: you may need to restart the kernel to use updated packages.


# After installation, you need to download a language model

In [22]:
!python -m spacy download en_core_web_md

/Users/thomasheigl/.pyenv/versions/3.10.6/envs/smart_food_diary/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 25.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [14]:
%pip install plac

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [1]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm

In [2]:
TRAIN_DATA = [
    ('Who is Nishanth?', {
        'entities': [(7, 15, 'PERSON')]
    }),
     ('Who is Kamal Khumar?', {
        'entities': [(7, 19, 'PERSON')]
    }),
    ('I like London and Berlin.', {
        'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]
    })
]

In [3]:
%pip install spacy-transformers
import spacy_transformers

Note: you may need to restart the kernel to use updated packages.


/Users/thomasheigl/.pyenv/versions/3.10.6/envs/smart_food_diary/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/thomasheigl/.pyenv/versions/3.10.6/envs/smart_food_diary/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/thomasheigl/.pyenv/versions/3.10.6/envs/smart_food_diary/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
from spacy.language import Language

In [13]:
nlp = spacy.blank('en')
tokens = nlp("Hello, I am Thomas and like bouldering")
tokens

spacy.tokens.doc.Doc

In [43]:
#load the model
#nlp = spacy.load('en_core_web_md', disable=["tagger", "ner"], exclude=["ner"])
nlp = spacy.blank('en')

print(nlp.pipe_names)
#set up the pipeline

# @Language.component("custom_ner")
# def my_component(doc):
#     return nlp.create_pipe('ner')
    #return EntityRecognizer(doc.vocab, labels=["PERSON", "LOC"])


#ner = nlp.create_pipe('ner')

ner = nlp.add_pipe('ner', last=True)
#ner = nlp.get_pipe('ner')

print(nlp.pipe_names, ner)

[]
['ner'] <spacy.pipeline.ner.EntityRecognizer object at 0x2823b1690>


In [94]:
for text, annot in tqdm(TRAIN_DATA):
    print(text, annot)

100%|██████████| 3/3 [00:00<00:00, 6849.71it/s]

Who is Nishanth? {'entities': [(7, 15, 'PERSON')]}
Who is Kamal Khumar? {'entities': [(7, 19, 'PERSON')]}
I like London and Berlin. {'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]}


In [42]:
import json

def load_data(path):
    with open(path, 'r') as f:
        data = json.load(f)
    return data

json = load_data("../data/food-diary-examples-classified.json")
TRAIN_DATA = [x for x in json['annotations'] if x is not None]
TRAIN_DATA

[['0.33 cola\r', {'entities': [[0, 4, 'QUANTITY'], [5, 9, 'LIQUID']]}],
 ['0.66ml Cola\r',
  {'entities': [[0, 4, 'QUANTITY'], [4, 6, 'UNIT'], [7, 11, 'LIQUID']]}],
 ['0.75L high C\r',
  {'entities': [[0, 4, 'QUANTITY'], [4, 5, 'UNIT'], [6, 12, 'LIQUID']]}],
 ['1 baked buns\r', {'entities': [[0, 1, 'QUANTITY'], [2, 12, 'FOOD']]}],
 ['1 bread with cheese 1 tea 0.4\r',
  {'entities': [[0, 1, 'QUANTITY'],
    [2, 19, 'FOOD'],
    [20, 21, 'QUANTITY'],
    [22, 25, 'LIQUID'],
    [26, 29, 'QUANTITY']]}],
 ['1 bread with cheese\r',
  {'entities': [[0, 1, 'QUANTITY'], [2, 19, 'FOOD']]}],
 ['1 bread with sausage 1 knoppers\r',
  {'entities': [[0, 1, 'QUANTITY'],
    [2, 20, 'FOOD'],
    [21, 22, 'QUANTITY'],
    [23, 31, 'FOOD']]}],
 ['1 bread with sausage 3 pick ups 1×tea\r',
  {'entities': [[0, 1, 'QUANTITY'],
    [2, 20, 'FOOD'],
    [21, 22, 'QUANTITY'],
    [23, 31, 'FOOD'],
    [32, 33, 'QUANTITY'],
    [34, 37, 'LIQUID']]}],
 ['1 bun with cream cheese\r',
  {'entities': [[0, 1, 'QUANTI

In [44]:
from spacy.training.example import Example

ner.add_label("FOOD")
ner.add_label("QUANTITY")
ner.add_label("UNIT")
ner.add_label("CALORIES")
ner.add_label("LIQUID")

#optimizer = nlp.create_optimizer() # Just use this if you create a blank new model. But here we use a pretrained model.
optimizer = nlp.begin_training()

losses = {}
#for batch in spacy.util.minibatch(TRAIN_DATA, size=2):
for text, annotations in TRAIN_DATA:
    print("Text:", text)
    print("Annotations:", annotations)
    # create Example
    example = Example.from_dict(nlp.make_doc(text), annotations)

    # Update the model
    nlp.update([example], drop=0.3, sgd=optimizer, losses=losses)

ner, losses

    # for itn in range(n_iter):
    #     random.shuffle(TRAIN_DATA)
    #     losses = {}
    #     for text, annotations in tqdm(TRAIN_DATA):
    #         nlp.update(
    #             [text],
    #             [annotations],
    #             drop=0.5,
    #             sgd=optimizer,
    #             losses=losses)
    #     print(losses)

Text: 0.33 cola
Annotations: {'entities': [[0, 4, 'QUANTITY'], [5, 9, 'LIQUID']]}
Text: 0.66ml Cola
Annotations: {'entities': [[0, 4, 'QUANTITY'], [4, 6, 'UNIT'], [7, 11, 'LIQUID']]}
Text: 0.75L high C
Annotations: {'entities': [[0, 4, 'QUANTITY'], [4, 5, 'UNIT'], [6, 12, 'LIQUID']]}
Text: 1 baked buns
Annotations: {'entities': [[0, 1, 'QUANTITY'], [2, 12, 'FOOD']]}
Text: 1 bread with cheese 1 tea 0.4
Annotations: {'entities': [[0, 1, 'QUANTITY'], [2, 19, 'FOOD'], [20, 21, 'QUANTITY'], [22, 25, 'LIQUID'], [26, 29, 'QUANTITY']]}
Text: 1 bread with cheese
Annotations: {'entities': [[0, 1, 'QUANTITY'], [2, 19, 'FOOD']]}
Text: 1 bread with sausage 1 knoppers
Annotations: {'entities': [[0, 1, 'QUANTITY'], [2, 20, 'FOOD'], [21, 22, 'QUANTITY'], [23, 31, 'FOOD']]}
Text: 1 bread with sausage 3 pick ups 1×tea
Annotations: {'entities': [[0, 1, 'QUANTITY'], [2, 20, 'FOOD'], [21, 22, 'QUANTITY'], [23, 31, 'FOOD'], [32, 33, 'QUANTITY'], [34, 37, 'LIQUID']]}
Text: 1 bun with cream cheese
Annotations

(<spacy.pipeline.ner.EntityRecognizer at 0x2823b1690>,
 {'ner': 994.56503022593})

In [45]:
for text, _ in TRAIN_DATA:
    print(f"Text:", text)
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Text: 0.33 cola
Entities [('0.33 cola', 'FOOD')]
Text: 0.66ml Cola
Entities [('0.66ml', 'QUANTITY'), ('Cola', 'FOOD')]
Text: 0.75L high C
Entities [('0.75L', 'QUANTITY'), ('high C', 'FOOD')]
Text: 1 baked buns
Entities [('1', 'QUANTITY'), ('baked buns', 'FOOD')]
Text: 1 bread with cheese 1 tea 0.4
Entities [('1', 'QUANTITY'), ('bread', 'FOOD'), ('cheese', 'FOOD'), ('1', 'QUANTITY'), ('tea', 'UNIT')]
Text: 1 bread with cheese
Entities [('1', 'QUANTITY'), ('bread', 'FOOD'), ('cheese', 'FOOD')]
Text: 1 bread with sausage 1 knoppers
Entities [('1', 'QUANTITY'), ('bread', 'FOOD'), ('1', 'QUANTITY'), ('knoppers', 'FOOD')]
Text: 1 bread with sausage 3 pick ups 1×tea
Entities [('1', 'QUANTITY'), ('bread', 'FOOD'), ('sausage', 'FOOD'), ('3', 'QUANTITY'), ('pick', 'UNIT')]
Text: 1 bun with cream cheese
Entities [('1', 'QUANTITY'), ('bun', 'FOOD')]
Text: 1 bun with cheese and jam and a boiled egg
Entities [('1', 'QUANTITY'), ('bun', 'FOOD'), ('cheese', 'FOOD'), ('jam', 'FOOD'), ('boiled', 'FOOD')

In [46]:
doc = nlp('Wonder chunks chicken style 185g, 1tbsp oil, All Seasons pan-fried vegetables Asian style approx 250g')
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Wonder chunks chicken', 'FOOD'), ('185', 'QUANTITY'), ('g', 'UNIT'), ('1tbsp', 'QUANTITY'), ('oil', 'FOOD'), ('All', 'UNIT'), ('pan-fried vegetables Asian style', 'FOOD'), ('250', 'QUANTITY'), ('g', 'UNIT')]


In [47]:
# from spacy.tokens import Span

# nlp = spacy.blank("en")

# # Create a Doc with entity spans
# doc1 = nlp("iPhone X is coming")
# doc1.ents = [Span(doc1, 0, 2, label="GADGET")]

# example = Example.from_dict(doc1, doc1.ents)
# nlp.update([example])

# # Create another doc without entity spans
# doc2 = nlp("I need a new phone! Any tips?")

# docs = [doc1, doc2]

# doc_result = nlp("iPhone X is coming")
# doc_result.ents


TypeError: Argument 'example_dict' has incorrect type (expected dict, got tuple)